# CNN model Using python
This note book demostrates the usage of basic python usage to create a CNN network for Geospatial data analysis.

# Import Library

In [ ]:
from osgeo import gdal
import geopandas as gdp
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble, model_selection

## Data info 
Following data are used in this exersize
- Raste Data : Landsat 8 data 
    We are using the stack images given in the folder data/LC08_L1TP_146039_20211229_20220106_01_T1
    -- LC08_L1TP_146039_20211229_20220106_01_T1_stack.tif 
        has 12 bands in the order (B1,B2,B3,B4,B5,B6,B7,B9,B10,B11,BQA). We will use the seven bands from B1-7 in this exersise.
- Vector data : 100 Sample points generate using the QGIS. 
   It contains the attributes 
   -- rand_point : Id of the random point generated
   -- Class : Integer represeting the classes (1:Snow/Clouds, 2:Waste Land , 3:Forest , 4: , 5: )

In [ ]:
img_path="./data/LC08_L1TP_146039_20211229_20220106_01_T1/LC08_L1TP_146039_20211229_20220106_01_T1_stack.tif"
point_path='./data/ClassPoints/rPoints/rPoints.shp'

In [ ]:
ds=gdal.Open(img_path)
trans=ds.GetGeoTransform()
data=ds.ReadAsArray()


In [ ]:
data.shape 


## Do the subsetting


In [ ]:
data=data[0:7,:,:]
data.shape

In [ ]:
data.shape
plt.imshow(data[1]) #plot the sencon basdn

In [ ]:
fig, ax = plt.subplots()
ax.plot(data[:,5000,2000]) #Check the seven bands details at location 5000,2000
trans=ds.GetGeoTransform()
Xloc = trans[1] * 5000 + trans[2] * 2000 + trans[0];
Yloc = trans[4] * 5000 + trans[5] * 2000 + trans[3];
ax.set_xticks([0, 1, 2 ,3 , 4 , 5 ,6])
labels=ax.set_xticklabels(['B1', 'B2', 'B3', 'B4', 'B5', 'B6','B7'])
ax.set_title(f"DN value at location ({Xloc} , {Yloc})")
ax.set_xlabel("Band No")
ax.set_ylabel("Band DN Value")

In [ ]:
df=gdp.read_file(point_path)

To get the sample value of points at location(lat,long) we need to use the following formula
- point.lon-raster.orgin.lon)/pixelsize
- point.lat-raster.orgin.lat)/pixelsize


In [ ]:
print(trans)

In [ ]:
def GetVal(X): #The function implements the above formual
    lon=X.geometry.x
    lat=X.geometry.y
    r=int((trans[3]-lat)/trans[1])
    c=int((trans[0]-lon)/trans[5])
    return data[k,r,c]

In [ ]:
noOfBands,_,_=data.shape #Get the number of bands in the data sets
print(noOfBands)
for k in range(0,noOfBands): #Get the bands values at sample location
    df["b"+str(k+1)]=df.apply(GetVal,axis='columns')

In [ ]:
df.head(3) #Print the details of data 

## Prepare the feautures and target 
For random Forest classfication from the sample points features and targets are created

In [ ]:
feature=df[['b1','b2','b3','b4','b5','b6','b7']].values #For sample point locatin  a feature vector contains the 
#informaion of only bands DN values
target=df['Class'].values #The class target label is subset

In [ ]:
print(target)

In [ ]:
target.shape #Confirm there are 100 sampel points
dataset=df[['b1','b2','b3','b4','b5','b6','b7','Class']]
dataset=dataset.to_numpy()
dataset.shape
print(dataset)

## Setup back Neural Network
We will do the following tasks to create a Neural network based Classification
- Initialise network
- Forward Propagate Computation
- Back Propagate Error Computation
- Train the Network
- Predict a subset area

In [ ]:
from random import seed
from random import random
from math import exp
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)

# Forward Propagate
It generates the output from a neural network by propagating an input through layer.
It requires the three steps.
- Neuron Activation.
- Neuron Transfer.
- Forward Propagation.


## Neuron Activation.
Neuron activation is calculated as the weighted sum of the inputs similar to regression sum.

$z^{(l)}= weightedSum = \sum(weight^{(l)} * x^{(l)}) + bias^{(l)}$

In [ ]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	weightedSum = weights[-1]
	for l in range(len(weights)-1):
		weightedSum += weights[l] * inputs[l]+weights[-1]
	return weightedSum

## Neuron Transfer
We can use different activation functions to transform the neuron activation. This includes 
1. $a^{(l)}(x)=\frac{1}{(1+e^{-𝑧^{(l)}(x)})}$    [Sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function)
2. $a^{(l)}(x)=max(0,𝑧^{(l)}(x))$  [Relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))
Here we are using the Sigmoid function

In [ ]:

def transfer(weightedSum):
	return 1.0 / (1.0 + exp(-weightedSum))


# Forward Propagation.

In [ ]:
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

## Back Propagate Error
1. Transfer Derivative.
- $a'^{(l)}=\frac{d}{dz}z^{(l)}=z^{(l)}(1-z^{(l)})$
2. Error Backpropagation.
- $\frac{\partial E}{\partial w^{(l)}}=\frac{\partial E}{\partial a^{(l)}}*\frac{\partial a^{(l)}}{\partial z^{(l)}}*\frac{ {\partial z^{(l)}}}{\partial w^{(l)}}$

In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

## Erro Propagate Error

In [ ]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


# Train Network

1. Update Weights.
- $w^{(l)}_{+} = w^{(l)} - \gamma * \frac{\partial E}{w^{(l)}} * input$. $\gamma$ is learningrate 
2. Train Network.

In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

## Train Network

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]-1] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [ ]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 18, n_outputs)
train_network(network, dataset, 0.5, 100, n_outputs)
for layer in network:
	print(layer)

# Predict

In [ ]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))+1

for row in dataset:
    predicted=predict(network,row)
    print(row,predicted)

# Applying for all pixel of image

In [ ]:
ouputdata=np.zeros((7911,7771)) #Data place to keep the classifed image data

In [ ]:
for r in range(7911):
    #print(f'processing row {r}')
    for c in range(7771):
        predictOn=data[:,r,c].reshape(-1)
        predictedValue=forward_propagate(network,predictOn)
        classValue=predictedValue.index(max(predictedValue))+1
        ouputdata[r,c]=classValue
        

In [ ]:
plt.imshow(ouputdata[2000:5050,6000:6050])
#print(ouputdata[4000:5050,4000:5050])